In [ ]:
# ce code ne marchera pas en l'etat 

In [ ]:
# IMPORT
import pandas as pd
import random
from random import randint
from difflib import SequenceMatcher
from math import floor

In [ ]:
# Fonctions 
def lev(s, t):
    m = len(s)
    n = len(t)
    d = [[0] * (n + 1) for i in range(m + 1)]  

    for i in range(1, m + 1):
        d[i][0] = i

    for j in range(1, n + 1):
        d[0][j] = j
    
    for j in range(1, n + 1):
        for i in range(1, m + 1):
            if s[i - 1] == t[j - 1]:
                cost = 0
            else:
                cost = 1
            d[i][j] = min(d[i - 1][j] + 1,      # deletion
                          d[i][j - 1] + 1,      # insertion
                          d[i - 1][j - 1] + cost) # substitution   

    return d[m][n]




# Function to calculate the 
# Jaro Similarity of two strings 
def jaro_distance(s1, s2) :

	# If the strings are equal 
	if (s1 == s2) :
		return 1.0; 

	# Length of two strings 
	len1 = len(s1);
	len2 = len(s2); 

	if (len1 == 0 or len2 == 0) :
		return 0.0; 

	# Maximum distance upto which matching 
	# is allowed 
	max_dist = (max(len(s1), len(s2)) // 2 ) - 1; 

	# Count of matches 
	match = 0; 

	# Hash for matches 
	hash_s1 = [0] * len(s1) ;
	hash_s2 = [0] * len(s2) ; 

	# Traverse through the first string 
	for i in range(len1) : 

		# Check if there is any matches 
		for j in range( max(0, i - max_dist), 
					min(len2, i + max_dist + 1)) : 
			
			# If there is a match 
			if (s1[i] == s2[j] and hash_s2[j] == 0) : 
				hash_s1[i] = 1; 
				hash_s2[j] = 1; 
				match += 1; 
				break; 
		
	# If there is no match 
	if (match == 0) :
		return 0.0; 

	# Number of transpositions 
	t = 0; 

	point = 0; 

	# Count number of occurrences 
	# where two characters match but 
	# there is a third matched character 
	# in between the indices 
	for i in range(len1) : 
		if (hash_s1[i]) :

			# Find the next matched character 
			# in second string 
			while (hash_s2[point] == 0) :
				point += 1; 

			if (s1[i] != s2[point]) :
				point += 1;
				t += 1;
			else :
				point += 1;
				
		t /= 2; 

	# Return the Jaro Similarity 
	return ((match / len1 + match / len2 +
			(match - t) / match ) / 3.0); 

# Jaro Winkler Similarity 
def jaro_Winkler(s1, s2) : 

	jaro_dist = jaro_distance(s1, s2); 

	# If the jaro Similarity is above a threshold 
	if (jaro_dist > 0.7) :

		# Find the length of common prefix 
		prefix = 0; 

		for i in range(min(len(s1), len(s2))) :
		
			# If the characters match 
			if (s1[i] == s2[i]) :
				prefix += 1; 

			# Else break 
			else :
				break; 

		# Maximum of 4 characters are allowed in prefix 
		prefix = min(4, prefix); 

		# Calculate jaro winkler Similarity 
		jaro_dist += 0.1 * prefix * (1 - jaro_dist); 

	return jaro_dist; 

# Driver code 
if __name__ == "__main__" : 

	s1 = "TRATE"; s2 = "TRACE"; 

	# Print Jaro-Winkler Similarity of two strings 
	print("Jaro-Winkler Similarity =", jaro_Winkler(s1, s2)) ; 

def lcs_dist(a,b):
    s = SequenceMatcher(a=a, b=b)
    return list(s.find_longest_match(0, len(a), 0, len(b)))[2]

def process_df(df, df_b, filter_conditions, score_func, proposition_col):
    i = 0
    for _, row in df.iterrows():
        df_b_filtered = df_b[filter_conditions(df_b, row)]
        if not df_b_filtered.empty:
            scores = df_b_filtered['rue'].apply(lambda x: score_func(str(x), str(row['rue'])))
            max_score_index = scores.idxmin() if score_func == lev else scores.idxmax()
            proposition_max_score = df_b_filtered.loc[max_score_index].copy()
            df.at[_, proposition_col] = proposition_max_score['rue']
            
        else:  
            df.at[_, proposition_col] = "rien"
            
        i += 1
        if i % 10 == 0:
            print(f"Processed {i} rows")
    return df

filter_conditions = lambda df_b, row: (
    (df_b['numero'] == row['numero']) &
    (df_b['rep'] == row['rep']) &
    (df_b['commune'] == row['commune'])&
    (df_b['libelle_commune'] == row['libelle_commune'])
    
)

filter_conditions_com = lambda df_b, row: (
    (df_b['commune'] == row['commune'])
)

In [ ]:
df_b.dtypes

In [ ]:
#df alleger 
PATH = "CEHDF_CLI_GEOCODAGE.csv"
df = pd.read_csv(PATH,encoding="UTF8")
df.head(3)

In [ ]:
df.columns
df.dtypes

In [ ]:
df["libelle_commune"] = df["LIBELLE_COMMUNE_BAN"]

In [ ]:
df.fillna('1', inplace=True)

In [ ]:
df_b.fillna('1', inplace=True)

In [ ]:
df_b.dtypes

In [ ]:
df_b["numero"] = df_b['numero']

In [ ]:
def process_df_optimized(df, df_b, filter_conditions, score_func, proposition_col):
    i = 0
    a = 0

    # Assurez-vous que les DataFrames ont des index réinitialisés
    df = df.reset_index(drop=True)
    df_b = df_b.reset_index(drop=True)

    # Appliquez la fonction filter_conditions à chaque ligne de df
    for index, row in df.iterrows():

        # Utilisez row directement pour obtenir des valeurs scalaires
        df_b_filtered = df_b[filter_conditions(df_b, row)]
        if not df_b_filtered.empty:
            # Appliquez la fonction score_func à chaque 'rue' dans df_b_filtered
            scores = df_b_filtered['rue'].apply(lambda x: score_func(str(x), str(row['rue'])))
            max_score_index = scores.idxmin() if score_func == lev else scores.idxmax()
            proposition_max_score = df_b_filtered.loc[max_score_index].copy()
            df.at[index, proposition_col] = proposition_max_score['rue']
        else:
            df.at[index, proposition_col] = "rien"
        i += 1
        if i % 100 == 0:
            print(f"Processed {i} rows")
        a = row['numero']
        b = row['rep']
        c = row['commune']
        d = row['libelle_commune']

    return df

# Définissez la fonction filter_conditions pour utiliser row directement
filter_conditions = lambda df_b, row: (
    (df_b['rep'] == row['rep']) &
    (df_b['commune'] == row['commune'])&
    (df_b['libelle_commune'] == row['libelle_commune'])
)



In [ ]:
from rapidfuzz.distance import Levenshtein
Levenshtein.distance("lewenstein", "levenshtein")

In [ ]:
df.sort_values(by=['rep',"commune","libelle_commune","numero"], inplace=True,
               ascending = [True, True, True , True])
print("Get the DataFrame after sorting:\n", df)

In [ ]:
keeps= ['rep', 'rue', 'commune', 'libelle_commune', 'numero']

In [ ]:
df = df[keeps]

In [ ]:
df_b.columns

In [ ]:
df_b = df_b[keeps]

In [ ]:
df_b.dtypes

In [ ]:
df.dtypes

In [ ]:
dataframe_BAN = df_b[['commune','rue','libelle_commune','numero','rep']].groupby(['commune','rue','libelle_commune','numero','rep']).first().reset_index()

In [ ]:
df_to_compute = df[['commune','rue','libelle_commune','numero','rep']].groupby(['commune','rue','libelle_commune','numero','rep']).first().reset_index()

In [ ]:
test = df_to_compute.merge(dataframe_BAN, on= ['commune','rep','libelle_commune','numero'] , how='left')

In [ ]:
test['rue_y'].isnull().unique()

In [ ]:
df_to_compute.dtypes

In [ ]:
dataframe_BAN.dtypes

In [ ]:
from rapidfuzz.distance import Levenshtein
Levenshtein.distance("lewenstein", "levenshtein")

In [ ]:
test.fillna('rien', inplace=True)

In [ ]:
teste = test[test["rue_y"]!= "rien"]

In [ ]:
teste

In [ ]:
test

In [ ]:
test['lev_distance'] = test.apply(lambda row: Levenshtein.distance(row['rue_x'], row['rue_y']), axis=1)

In [ ]:
idx = test.groupby(['rue_x', 'commune','rep','libelle_commune','numero'])['lev_distance'].idxmin()

# Utiliser l'index pour obtenir les lignes correspondantes du DataFrame
df_resultat = test.loc[idx]

In [ ]:
df_resultat

In [ ]:
i=0
for _, row in df_resultat.iterrows():
    a = row['rue_x']
    b= row['rue_y']
    s = SequenceMatcher(a=a, b=b)


           
    df_resultat.at[_, 'lcs'] = list(s.find_longest_match(0, len(a), 0, len(b)))[2]

In [ ]:
for _, row in df_resultat.iterrows():

  
     # Faire la proposition avec le score le plus élevé
    if not row["lcs"]==0:

            # Ajouter la proposition à la colonne 'proposition' dans df
        df_resultat.at[_, 'lcs_ratio1'] = row["lcs"]/len(row["rue_x"])


    else:  
        df_resultat.at[_, 'lcs_ratio1'] = 0

In [ ]:
for _, row in df_resultat.iterrows():

  
     # Faire la proposition avec le score le plus élevé
    if not row["lcs"]==0:

            # Ajouter la proposition à la colonne 'proposition' dans df
        df_resultat.at[_, 'lcs_ratio2'] = row["lcs"]/len(row["rue_y"])


    else:  
        df_resultat.at[_, 'lcs_ratio2'] = 0

In [ ]:
i=0
for _, row in df_resultat.iterrows():

     # Faire la proposition avec le score le plus élevé
    if not row["lcs"]==0:


   

            # Ajouter la proposition à la colonne 'proposition' dans df
        df_resultat.at[_, 'jaro-winkler'] = jaro_Winkler(row["rue_x"],row["rue_y"])

            # Ajouter le score correspondant à la colonne 'score_max' dans df
        
        
        
       
    else:  
        df_resultat.at[_, 'jaro-winkler'] = 0

            # Ajouter le score correspondant à la colonne 'score_max' dans df
        df_resultat.at[_, 'score_min'] = 10000
 

In [ ]:
df_resultat

In [ ]:
#df.to_csv("sample_rajout.csv")

In [ ]:
#df.to_csv("RESULTAT_SAMPLE2.csv")

In [ ]:
#dff3.to_csv("RESULTAT_SAMPLE_JAROWINKLER.csv")

In [ ]:
df_resultat.to_csv("df_avec_dist_lev_prop")